In [ ]:
import json
import pandas as pd
import numpy as np

chosen_state = "south_carolina"
chosen_state_abbr = "sc"
state_districts = 7

summary_filepath = f"states/{chosen_state}/congressional_districts/summary/{chosen_state_abbr}_congressional_districts_summary_seawulf.json"
with open(summary_filepath, 'r') as summary_file_r:
    summary_json = json.load(summary_file_r)

precincts_economics_filepath = f"states/{chosen_state}/economic/{chosen_state}_precincts_household_income.json"
with open(precincts_economics_filepath, 'r') as precincts_income_file:
    precincts_income_json = json.load(precincts_income_file)

district_precincts_filepath = f"states/{chosen_state}/geodata/{chosen_state}_precincts.geojson"
with open(district_precincts_filepath, 'r') as district_precincts_file:
    district_precincts_json = json.load(district_precincts_file)

# Map precincts to districts
district_precincts = {
    str(i).zfill(2): []
    for i in range(1, state_districts+1)
}

features = district_precincts_json['features']
for feature in features:
    props = feature['properties']
    precinct_id = props['UNIQUE_ID']
    district_id = props['CONG_DIST']
    district_precincts[district_id].append(precinct_id)

for district_id, precincts in district_precincts.items():
    precinct_households = []
    precinct_households_0_35K = []
    precinct_households_35K_60K = []
    precinct_households_60K_100K = []
    precinct_households_100K_125K = []
    precinct_households_125K_150K = []
    precinct_households_150K_MORE = []
    for precinct_id in precincts:
        located_precinct = None
        for precinct in precincts_income_json:
            if precinct["UNIQUE_ID"] == precinct_id:
              located_precinct = precinct
              break
        precinct_households_0_35K.append(located_precinct["0_35K"])
        precinct_households_35K_60K.append(located_precinct["35K_60K"])
        precinct_households_60K_100K.append(located_precinct["60K_100K"])
        precinct_households_100K_125K.append(located_precinct["100K_125K"])
        precinct_households_125K_150K.append(located_precinct["125K_150K"])
        precinct_households_150K_MORE.append(located_precinct["150K_MORE"])
        precinct_households.append(located_precinct["TOT_HOUS22"])

    district_households_count = int(sum(precinct_households))
    district_0_35K_count = (sum(precinct_households_0_35K))
    percent_0_35K = round((district_0_35K_count / district_households_count) * 100)

    district_35K_60K_count = (sum(precinct_households_35K_60K))
    percent_35K_60K = round((district_35K_60K_count / district_households_count) * 100)

    district_60K_100K_count = (sum(precinct_households_60K_100K))
    percent_60K_100K = round((district_60K_100K_count / district_households_count) * 100)

    district_100K_125K_count = (sum(precinct_households_100K_125K))
    percent_100K_125K = round((district_100K_125K_count / district_households_count) * 100)

    district_125K_150K_count = (sum(precinct_households_125K_150K))
    percent_125K_150K = round((district_125K_150K_count / district_households_count) * 100)

    district_150K_MORE_count = (sum(precinct_households_150K_MORE))
    percent_150K_MORE = round((district_150K_MORE_count / district_households_count) * 100)

    rounded_sum = percent_0_35K + percent_35K_60K + percent_60K_100K + percent_100K_125K + percent_125K_150K + percent_150K_MORE

    percentages = {
      "0_35K_PERCENT": percent_0_35K,
      "35K_60K_PERCENT": percent_35K_60K,
      "60K_100K_PERCENT": percent_60K_100K,
      "100K_125K_PERCENT": percent_100K_125K,
      "125K_150K_PERCENT": percent_125K_150K,
      "150K_MORE_PERCENT": percent_150K_MORE
    }
    # print("Before:")
    # print(percentages, rounded_sum)

    if rounded_sum == 99 or rounded_sum == 101:
      decimal_portions = {}
      for economic_group, percentage in percentages.items():
        # Calculate decimal portion by subtracting the int part from percentage
        decimal_value = percentage - int(percentage)
        # Store the decimal portion in dict with corresponding region type
        decimal_portions[economic_group] = decimal_value
        decimal_region = None
      if rounded_sum == 99: 
        decimal_region = max(decimal_portions, key=decimal_portions.get)
        percentages[decimal_region] += 1
      if rounded_sum == 101: 
        decimal_region = min(decimal_portions, key=decimal_portions.get)
        percentages[decimal_region] -= 1


    # for index in range(0, state_districts):
    #   if str(summary_json["data"]["CONG_DIST"]).zfill(2) == district_id:
    #     for percent_key, percent_value in percentages.items():
    #       summary_json["data"][index][percent_key] = percent_value
    #     break

    index = 0
    # print(summary_json["data"][3])
    for district_data in summary_json["data"]:
      if str(district_data["CONG_DIST"]).zfill(2) == district_id:
        for percent_key, percent_value in percentages.items():
          summary_json["data"][index][percent_key] = percent_value
        break
      else:
        index += 1

finalized_json_str = json.dumps(summary_json, indent=4)
with open(summary_filepath, 'w') as summary_file_w:
    summary_file_w.write(finalized_json_str)

